In [42]:
# import all packages needed
import string 
import numpy as np
import pandas as pd
from matplotlib import pyplot
from base64 import b64decode as decode
import math
import torch
import torch.nn as nn
from torch.fft import fftn
import torch.nn.functional as F
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification

from sklearn.model_selection import train_test_split

## Data Processing / Cleaning

In [2]:
# use class base64 to decode waveform data
def to_array(wf):
    barr = bytearray(decode(wf))
    vals = np.array(barr)
    return vals.view(np.int16)

# read in data
exam_data = pd.read_csv("data/d_exam.csv").drop(columns = ["site_num", "patient_id_edit"])
waveform_data = pd.read_csv("data/d_waveform.csv")
lead_data = pd.read_csv("data/d_lead_data.csv").drop(columns = ["exam_id"])
diagnosis_data = pd.read_csv("data/d_diagnosis.csv").drop(columns = ["user_input"])

# add decoded data as a column to lead data
waveforms = list(lead_data['waveform_data'])
lead_data['decoded_waveform'] = [to_array(i) for i in waveforms]

# merge waveform data and lead data
waveform_lead = lead_data.merge(waveform_data, how = "left", left_on = "waveform_id", right_on = "waveform_id", suffixes = (None, None))

#  sort by exam id and lead id
waveform_lead.sort_values(by = ["waveform_id", "lead_id"], inplace = True)

waveform_lead.loc[:, ['exam_id', 'lead_id', 'decoded_waveform', 'waveform_type']]


# adding the diagnosis and labels
waveform_and_diag = pd.merge(waveform_lead[['exam_id', 'lead_id', 'decoded_waveform', 'waveform_type']], diagnosis_data[["exam_id", "Full_text", "Original_Diag"]], left_on= "exam_id", right_on="exam_id")
waveform_and_diag

,exam_id,lead_id,decoded_waveform,waveform_type,Full_text,Original_Diag
0,549871,I,"[-8, -8, -8, -8, -8, -8, -8, -7, -6, -5, -4, -...",Rhythm,No previous ECGs available,0
1,549871,I,"[-8, -8, -8, -8, -8, -8, -8, -7, -6, -5, -4, -...",Rhythm,Otherwise normal ECG,0
2,549871,I,"[-8, -8, -8, -8, -8, -8, -8, -7, -6, -5, -4, -...",Rhythm,Sinus bradycardia,0
3,549871,I,"[-8, -8, -8, -8, -8, -8, -8, -7, -6, -5, -4, -...",Rhythm,NaN,0
4,549871,I,"[-8, -8, -8, -8, -8, -8, -8, -7, -6, -5, -4, -...",Rhythm,Sinus bradycardia,1
...,...,...,...,...,...,...
1419,554080,V6,"[10, 10, 10, 11, 12, 12, 12, 12, 12, 13, 14, 1...",Rhythm,Abnormal ECG,1
1420,554080,V6,"[10, 10, 10, 11, 12, 12, 12, 12, 12, 13, 14, 1...",Rhythm,No previous ECGs available,1
1421,554080,V6,"[10, 10, 10, 11, 12, 12, 12, 12, 12, 13, 14, 1...",Rhythm,No previous ECGs available,0
1422,554080,V6,"[10, 10, 10, 11, 12, 12, 12, 12, 12, 13, 14, 1...",Rhythm,Sinus rhythm,0


In [3]:
# concatenate all leads into a single array
waveform_lead_concat = waveform_lead.groupby(["exam_id", "waveform_type"])['decoded_waveform'].apply(lambda x: tuple(x)).reset_index()

# remove irregular observations, concat tuple into numpy array
waveform_lead_concat = waveform_lead_concat.drop([12,17], axis = 0)
waveform_lead_concat['decoded_waveform'] = waveform_lead_concat['decoded_waveform'].apply(lambda x: np.vstack(x))

waveform_lead_rhythm = waveform_lead_concat[waveform_lead_concat['waveform_type'] == "Rhythm"]
waveform_lead_median = waveform_lead_concat[waveform_lead_concat['waveform_type'] == "Median"]

waveform_lead_rhythm

,exam_id,waveform_type,decoded_waveform
1,548759,Rhythm,"[[4, 3, 2, -1, -4, -4, -4, -4, -4, -7, -10, -8..."
3,549871,Rhythm,"[[-8, -8, -8, -8, -8, -8, -8, -7, -6, -5, -4, ..."
5,550602,Rhythm,"[[-22, -20, -18, -16, -14, -14, -14, -12, -10,..."
7,551485,Rhythm,"[[46, 45, 44, 42, 40, 35, 30, 26, 22, 18, 14, ..."
9,552077,Rhythm,"[[-7, -4, -1, -6, -10, -12, -14, -11, -11, -14..."
11,552856,Rhythm,"[[-32, -32, -32, -33, -34, -34, -34, -33, -32,..."
14,553115,Rhythm,"[[-8, -5, -2, -2, -2, -5, -8, -8, -8, -7, -6, ..."
16,553528,Rhythm,"[[-12, -12, -12, -12, -12, -12, -12, -12, -12,..."


In [4]:
# Adding the labels/sentences
exams = diagnosis_data["exam_id"].unique()

# Let's look over this tomorrow
diagnosis_data = diagnosis_data[diagnosis_data['Original_Diag'] == 1].dropna()
searchfor = ['previous', 'unconfirmed', 'compared', 'interpretation', 'significant']
diagnosis_data = diagnosis_data.loc[diagnosis_data['Full_text'].str.contains('|'.join(searchfor)) != 1]
#

diagnosis_data.sort_values(by=["exam_id", "statement_order"], inplace=True)
diagnoses = []
curr_id = 0
curr_string = ""
for i, row in diagnosis_data.iterrows():
    if row["statement_order"] == 1 and curr_string != "":
        curr_string = curr_string.lower().translate(str.maketrans('', '', string.punctuation))
        val = [curr_id, curr_string[1:]]
        diagnoses.append(val)
        curr_string = ""
        curr_id = row["exam_id"]

    if curr_id == 0:
        curr_id = row["exam_id"]
    
    curr_string += " " + row["Full_text"]

diagnosis_df = pd.DataFrame(diagnoses, columns = ['exam_id', 'diagnosis'])
waveform_lead_rhythm_diag = pd.merge(left=waveform_lead_rhythm, right=diagnosis_df, left_on='exam_id', right_on='exam_id')

#waveform_lead_rhythm_diag
waveform_lead_rhythm_diag

,exam_id,waveform_type,decoded_waveform,diagnosis
0,548759,Rhythm,"[[4, 3, 2, -1, -4, -4, -4, -4, -4, -7, -10, -8...",normal sinus rhythm low voltage qrs borderline...
1,549871,Rhythm,"[[-8, -8, -8, -8, -8, -8, -8, -7, -6, -5, -4, ...",sinus bradycardia otherwise normal ecg
2,550602,Rhythm,"[[-22, -20, -18, -16, -14, -14, -14, -12, -10,...",sinus tachycardia otherwise normal ecg
3,551485,Rhythm,"[[46, 45, 44, 42, 40, 35, 30, 26, 22, 18, 14, ...",normal sinus rhythm normal ecg
4,552077,Rhythm,"[[-7, -4, -1, -6, -10, -12, -14, -11, -11, -14...",normal sinus rhythm normal ecg
5,552856,Rhythm,"[[-32, -32, -32, -33, -34, -34, -34, -33, -32,...",normal sinus rhythm with sinus arrhythmia mini...
6,553115,Rhythm,"[[-8, -5, -2, -2, -2, -5, -8, -8, -8, -7, -6, ...",atrial fibrillation abnormal ecg normal sinus ...


In [5]:
unique_words = set()
for num, sentence in diagnoses:
    for word in sentence.split():
        unique_words.add(word)
print(unique_words)

{'normal', 'consider', 'ischemia', 'with', 'variant', 'inferior', 'may', 'criteria', 'tachycardia', 'abnormal', 'borderline', 'wave', 'voltage', 'for', 't', 'qrs', 'otherwise', 'arrhythmia', 'abnormality', 'atrial', 'bradycardia', 'be', 'ecg', 'fibrillation', 'sinus', 'lvh', 'rhythm', 'minimal', 'low'}


In [6]:
# split data into training and testing datasets
# y not included for now
def one_hot(x, dict_words, max_length):
    x = x.split(" ")
    array = []
    for i in x:
        array.append(dict_words.index(i))
    while(len(array) < max_length):
        array.append(29)
    return array

dict_words = list(unique_words)
dict_words.append([" "])
print(len(dict_words))
Y = waveform_lead_rhythm_diag['diagnosis'].apply(lambda x: one_hot(x, dict_words, 20))

train_x, test_x, train_y, test_y = train_test_split(waveform_lead_rhythm_diag['decoded_waveform'], Y, test_size = 0.1, random_state = 2021)
train_x = torch.tensor(list(train_x)).float()
train_y = torch.tensor(list(train_y))

test_x = torch.tensor(list(test_x)).float()
test_y = torch.tensor(list(test_y))

30


## Model 1 - Conv1D Encoder w/ LSTM Decoder

In [ ]:
# HYPERPARAMETERS
J = 10 # max number of filters per class
LR = 1e-3

# define global max pooling
class global_max_pooling_1d(nn.Module):
    def __init__(self):
        super().__init__()
    
    def forward(self, x):
        x, _ = torch.max(x, dim = 2)
        return(x)

# 1D grouped encoder model
encoder_conv = nn.Sequential()
encoder_conv.add_module('initial_norm', nn.BatchNorm1d(8))
encoder_conv.add_module('conv_1', nn.Conv1d(in_channels = 8, out_channels = 8, groups = 8, kernel_size = 5, padding = 2))
for i in range(2, (J+2), 2):
    if (i-2) == 0: 
        prev = 8
    else:
        prev = (i-2)*8
    encoder_conv.add_module('conv_{num}'.format(num = int(i / 2 + 1)), nn.Conv1d(in_channels = prev, out_channels = i*8, groups = 8, kernel_size = 5, padding = 2, stride = 2))
    encoder_conv.add_module('activation_{num}'.format(num = int(i / 2 + 1)), nn.ELU())
    encoder_conv.add_module('batch_norm_{num}'.format(num = int(i / 2 + 1)), nn.BatchNorm1d(i*8))
    
encoder_conv.add_module('final_conv', nn.Conv1d(in_channels = J * 8, out_channels = 8, groups = 8, kernel_size = 5, padding = 2))
encoder_conv.add_module('max_pool', nn.MaxPool1d(kernel_size = 7, padding = 3, stride = 4))
encoder_conv.add_module('reshape', nn.MaxPool1d(kernel_size = 5, padding = 2, stride = 1))


# summarize model, verify output is of desired shape
print(encoder_conv)
print(encoder_conv(train_x).shape)

In [55]:
# HYPERPARAMETERS
J = 4 # max number of filters per class
LR = 1e-3

# define global max pooling
class global_max_pooling_1d(nn.Module):
    def __init__(self):
        super().__init__()
    
    def forward(self, x):
        x, _ = torch.max(x, dim = 2)
        return(x)

# 1D grouped encoder model
encoder_conv = nn.Sequential()
encoder_conv.add_module('initial_norm', nn.BatchNorm1d(8))
encoder_conv.add_module('conv_1', nn.Conv1d(in_channels = 8, out_channels = 8, groups = 8, kernel_size = 5, padding = 4, stride = 1))
for i in range(2, (J+2), 2):
    if (i-2) == 0: 
        prev = 8
    else:
        prev = (i-2)*8
    encoder_conv.add_module('conv_{num}'.format(num = int(i / 2 + 1)), nn.Conv1d(in_channels = prev, out_channels = i*8, groups = 8, kernel_size = 5, padding = 2, stride = 2))
    encoder_conv.add_module('activation_{num}'.format(num = int(i / 2 + 1)), nn.ELU())
    encoder_conv.add_module('batch_norm_{num}'.format(num = int(i / 2 + 1)), nn.BatchNorm1d(i*8))
    
encoder_conv.add_module('final_conv', nn.Conv1d(in_channels = J * 8, out_channels = 8, groups = 8, kernel_size = 5, padding = 2))
encoder_conv.add_module('max_pool', nn.MaxPool1d(kernel_size = 5, padding = 2, stride = 1))
#encoder_conv.add_module('reshape', nn.MaxPool1d(kernel_size = 5, padding = 2, stride = 1))

# summarize model, verify output is of desired shape
print(encoder_conv)
print(encoder_conv(train_x).shape)

Sequential(
  (initial_norm): BatchNorm1d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv_1): Conv1d(8, 8, kernel_size=(5,), stride=(1,), padding=(4,), groups=8)
  (conv_2): Conv1d(8, 16, kernel_size=(5,), stride=(2,), padding=(2,), groups=8)
  (activation_2): ELU(alpha=1.0)
  (batch_norm_2): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv_3): Conv1d(16, 32, kernel_size=(5,), stride=(2,), padding=(2,), groups=8)
  (activation_3): ELU(alpha=1.0)
  (batch_norm_3): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (final_conv): Conv1d(32, 8, kernel_size=(5,), stride=(1,), padding=(2,), groups=8)
  (max_pool): MaxPool1d(kernel_size=5, stride=1, padding=2, dilation=1, ceil_mode=False)
)
torch.Size([6, 8, 626])


## Model 2 - LSTM Encoder w/ Huggingface Decoder

In [10]:
# define hyperparameters 
hidden_layers = 512
embedding_dim = 8
num_words = len(dict_words)

class ECG_LSTM(nn.Module):
    def __init__(self, h_dim, e_dim, word_list_length):
        super(ECG_LSTM, self).__init__()
        # LSTM decoder  
        self.lstm = nn.LSTM(e_dim, h_dim)
        self.linear = nn.Linear(h_dim, word_list_length)
        
    def forward(self, seq):
        seq_embedded = seq.view(len(seq), -1, embedding_dim)
        final_hidd, _ = self.lstm(seq_embedded)
        dec_seq = self.linear(final_hidd)
        return F.log_softmax(dec_seq, dim = 1)
    

In [80]:
epoch = 1000
loss_fn = nn.NLLLoss()
optimizer = torch.optim.Adam(lstm_mod.parameters(), lr = 1e-3)
torch.autograd.set_detect_anomaly(True)

for i in range(epoch):
    for j, k in zip(train_x, train_y):
        optimizer.zero_grad()
        outputs = lstm_mod(j.unsqueeze(0)).squeeze(0)
        loss = loss_fn(outputs, k)
        loss.backward(retain_graph=True)
        optimizer.step()


KeyboardInterrupt: 

In [81]:
#torch.save(lstm_mod.state_dict(), 'model/lstm.pt')
lstm_mod = ECG_LSTM(encoder_conv, hidden_layers, embedding_dim, num_words)
lstm_mod.load_state_dict(torch.load('model/lstm.pt'))

out = lstm_mod(train_x[5].unsqueeze(0))
print(out.squeeze(0).detach().numpy().shape)
out = np.argmax(out.squeeze(0).detach().numpy(), axis = 1)
print(out)
print(train_y[5])

(20, 30)
[21 11 10 21  5 26 23 25 22  6 29 25 27 26  4 26  4 26 26  7]
tensor([21, 11, 10, 21,  5, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29,
        29, 29])


## Model 3 - Basic Transformer Architecture with Multi-Head Attention

## Model 4 - FNET Transformer Architecture

In [59]:
class FeedForwardNet(nn.Module):
    def __init__(self, features, expansion, dropout):
        super(FeedForwardNet, self).__init__()
        self.linear_1 = nn.Linear(features, features * expansion)
        self.linear_2 = nn.Linear(features * expansion, features)
        self.dropout_1 = nn.Dropout(dropout)
        self.dropout_2 = nn.Dropout(dropout)
        self.norm_1 = nn.LayerNorm(features)

    def forward(self, x):
        res = x
        x = F.gelu(self.linear_1(x))
        x = self.dropout_1(x)
        x = self.dropout_2(self.linear_2(x))
        x = self.norm_1(x + res)
        return x
    
class PositionalEncoding(nn.Module):
    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x):
        """
        Args:
            x: Tensor, shape [seq_len, batch_size, embedding_dim]
        """
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)
    
class FNETLayer(nn.Module):
    def __init__(self, features, expansion, dropout):
        super(FNETLayer, self).__init__()
        self.feed_forward = FeedForwardNet(features, expansion, dropout)
        self.norm_1 = nn.LayerNorm(features)
    
    def forward(self, x):
        res = x
        x = fftn(x, dim = (-1, -2)).real
        x = self.norm_1(x + res)
        x = self.feed_forward(x)
        return x
    
class FNETEncoder(nn.TransformerEncoder):
    def __init__(self, features, expansion=2, dropout=0.5, num_layers=6):
        encoder_layer = FNETLayer(features, expansion, dropout)
        super().__init__(encoder_layer=encoder_layer, num_layers=num_layers)

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

class FNETModel(nn.Module):
    def __init__(self, expansion, dropout, num_layers, embedder, decoder):
        super(FNETModel, self).__init__()
        self.embedder = embedder
        self.decoder = decoder
        temp = self.embedder(train_x).shape[2]
        self.pos_enb = PositionalEncoding(d_model = temp)
        self.encoder = FNETEncoder(features = temp, expansion = expansion, dropout = dropout, num_layers = num_layers)
        
    
    def forward(self, x):
        x = self.embedder(x)
        x = self.pos_enb(x)
        x = self.encoder(x)
        out = self.decoder(x)
        return out    

class Transpose(nn.Module):
    def __init__(self):
        super().__init__()
    def forward(self, x):
        return x.transpose(1, 2)

decoder = nn.Sequential(Transpose(), nn.Linear(8, 15), nn.ReLU(), nn.Linear(15, 20), nn.ReLU(), Transpose(), nn.Linear(626, 400), nn.ReLU(), nn.Linear(400, 200), nn.ReLU(), nn.Linear(200, 100), nn.ReLU(), nn.Linear(100, 30))

In [60]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
epoch = 300
#ECG_LSTM(hidden_layers, embedding_dim, num_words)
model = FNETModel(expansion = 2, dropout = 0.1, num_layers = 8, embedder = encoder_conv, decoder = decoder)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-3)
torch.autograd.set_detect_anomaly(True)

for i in range(epoch):
    losses = 0
    for j, k in zip(train_x, train_y):
        optimizer.zero_grad()
        outputs = model(j.unsqueeze(0)).squeeze(0)
        loss = loss_fn(outputs, k)
        losses += loss
    losses.backward(retain_graph=True)
    optimizer.step()
    print(losses)


tensor(20.0516, grad_fn=<AddBackward0>)
tensor(19.7664, grad_fn=<AddBackward0>)
tensor(19.3583, grad_fn=<AddBackward0>)
tensor(18.6789, grad_fn=<AddBackward0>)
tensor(17.6058, grad_fn=<AddBackward0>)
tensor(16.2079, grad_fn=<AddBackward0>)
tensor(14.6231, grad_fn=<AddBackward0>)
tensor(13.4965, grad_fn=<AddBackward0>)
tensor(13.2718, grad_fn=<AddBackward0>)
tensor(13.2807, grad_fn=<AddBackward0>)
tensor(13.0155, grad_fn=<AddBackward0>)
tensor(12.4728, grad_fn=<AddBackward0>)
tensor(12.0347, grad_fn=<AddBackward0>)
tensor(11.6499, grad_fn=<AddBackward0>)
tensor(11.4429, grad_fn=<AddBackward0>)
tensor(11.3366, grad_fn=<AddBackward0>)
tensor(11.2243, grad_fn=<AddBackward0>)
tensor(10.8664, grad_fn=<AddBackward0>)
tensor(10.6123, grad_fn=<AddBackward0>)
tensor(10.4745, grad_fn=<AddBackward0>)
tensor(10.3840, grad_fn=<AddBackward0>)
tensor(10.1632, grad_fn=<AddBackward0>)
tensor(10.0592, grad_fn=<AddBackward0>)
tensor(9.9358, grad_fn=<AddBackward0>)
tensor(9.8227, grad_fn=<AddBackward0>)
te

tensor(0.1467, grad_fn=<AddBackward0>)
tensor(0.1447, grad_fn=<AddBackward0>)
tensor(0.1435, grad_fn=<AddBackward0>)
tensor(0.1454, grad_fn=<AddBackward0>)
tensor(0.1421, grad_fn=<AddBackward0>)
tensor(0.1438, grad_fn=<AddBackward0>)
tensor(0.1435, grad_fn=<AddBackward0>)
tensor(0.1428, grad_fn=<AddBackward0>)
tensor(0.1421, grad_fn=<AddBackward0>)
tensor(0.1419, grad_fn=<AddBackward0>)
tensor(0.1421, grad_fn=<AddBackward0>)
tensor(0.1416, grad_fn=<AddBackward0>)
tensor(0.1420, grad_fn=<AddBackward0>)
tensor(0.1418, grad_fn=<AddBackward0>)
tensor(0.1422, grad_fn=<AddBackward0>)
tensor(0.1411, grad_fn=<AddBackward0>)
tensor(0.1413, grad_fn=<AddBackward0>)
tensor(0.1415, grad_fn=<AddBackward0>)
tensor(0.1407, grad_fn=<AddBackward0>)
tensor(0.1427, grad_fn=<AddBackward0>)
tensor(0.1411, grad_fn=<AddBackward0>)
tensor(0.1421, grad_fn=<AddBackward0>)
tensor(0.1430, grad_fn=<AddBackward0>)
tensor(0.1401, grad_fn=<AddBackward0>)
tensor(0.1403, grad_fn=<AddBackward0>)
tensor(0.1402, grad_fn=<A

In [63]:
torch.save(model.state_dict(), 'model/fnet.pt')
model = FNETModel(expansion = 2, dropout = 0.1, num_layers = 8, embedder = encoder_conv, decoder = decoder)
model.load_state_dict(torch.load('model/fnet.pt'))

out = (model(train_x[1].unsqueeze(0)))
print(out.squeeze(0).detach().numpy().shape)
out = np.argmax(out.squeeze(0).detach().numpy(), axis = 1)
print(out)
print(train_y[1])

(20, 30)
[20 16 18 23 21 24 10 25 24  1 21 23 29 29 29 29 29 29 29 29]
tensor([20, 16, 18, 23, 21, 24, 10, 25, 24,  1, 21, 23, 29, 29, 29, 29, 29, 29,
        29, 29])


In [44]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2Model.from_pretrained('gpt2')

#encoded_inputs = tokenizer(list(waveform_lead_rhythm_diag['diagnosis'])[0], return_tensors='pt')

encoded_inputs = tokenizer('Hello my name is Daniel', return_tensors = 'pt')
print(model)

GPT2Model(
  (wte): Embedding(50257, 768)
  (wpe): Embedding(1024, 768)
  (drop): Dropout(p=0.1, inplace=False)
  (h): ModuleList(
    (0): GPT2Block(
      (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (attn): GPT2Attention(
        (c_attn): Conv1D()
        (c_proj): Conv1D()
        (attn_dropout): Dropout(p=0.1, inplace=False)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
      (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (mlp): GPT2MLP(
        (c_fc): Conv1D()
        (c_proj): Conv1D()
        (dropout): Dropout(p=0.1, inplace=False)
      )
    )
    (1): GPT2Block(
      (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (attn): GPT2Attention(
        (c_attn): Conv1D()
        (c_proj): Conv1D()
        (attn_dropout): Dropout(p=0.1, inplace=False)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
      (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (mlp): GPT2MLP

## Model 5 - FNET/Basic Mixup Architecture 

In [45]:
from transformers import BertTokenizer, BertForNextSentencePrediction
import torch

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForNextSentencePrediction.from_pretrained('bert-base-uncased')

prompt = "In Italy, pizza served in formal settings, such as at a restaurant, is presented unsliced."
next_sentence = "The sky is blue due to the shorter wavelength of blue light."
encoding = tokenizer(prompt, next_sentence, return_tensors='pt')

outputs = model(**encoding, labels=torch.LongTensor([1]))
logits = outputs.logits
assert logits[0, 0] < logits[0, 1] # next sentence was random

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForNextSentencePrediction: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForNextSentencePrediction from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForNextSentencePrediction from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
